In [1]:
import pyspark
df=sqlContext.sql("select * from customer_churn_csv")

In [2]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [3]:
df.columns

Out[ 4 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [4]:
data=df.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites','Churn'])

In [5]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

data_assembled=assembler.transform(data)
dat=data_assembled.select(['features','Churn'])

In [6]:
train,test=dat.randomSplit([0.7,0.3])

In [7]:
from pyspark.ml.classification import LogisticRegression

In [8]:
model=LogisticRegression(labelCol='Churn')

In [9]:
fitted=model.fit(test)

In [10]:
final=fitted.transform(test)
pred_and_labels = fitted.evaluate(test)

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [12]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='Churn')

In [13]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [14]:
auc

Out[ 36 ]: 0.8305151706700379